# C2SIM

In [15]:
import jax
import chex
import ollama
from jax import numpy as jnp, jit, vmap, random
from jaxmarl import make
from jaxmarl.environments.smax import map_name_to_scenario

from einops import rearrange
from functional import partial
from tqdm import tqdm
from typing import Callable, Any, Tuple, Dict, List
import yaml

from src import plot_fn, make_bt

In [16]:
# globals
model = "mistral:7b-instruct-q2_K"
n_envs = 100
n_steps = 100
bt_path = "data/bt.yaml"
scenario = "3s5z_vs_3s6z"
num_allies = 10
num_enemies = 10

## language model

## smax trajectories

In [17]:
def step_fn(bt, rng, old_state_v, obs_v, env):  # take a step in the env
    rng, act_rng, step_rng = random.split(rng, 3)
    act_keys = random.split(act_rng, env.num_agents * n_envs).reshape(-1, n_envs, 2)
    step_keys = random.split(step_rng, n_envs)
    acts = {a: bt(act_keys[i], obs_v[a], a) for i, a in enumerate(env.agents)}  # para
    obs_v, state_v, reward_v, _, _ = vmap(env.step)(step_keys, old_state_v, acts)
    return obs_v, (bt, rng, state_v), (step_keys, old_state_v, acts), reward_v


def traj_fn(bt, rng, env, state_seq=[], reward_seq=[]):  # take n_steps in m env
    rng, reset_rng = random.split(random.PRNGKey(0))  # split rng for reset and step
    reset_keys = random.split(reset_rng, n_envs)  # split reset rng for n_envs

    step = partial(step_fn, env=env)  # partial step function
    obs_v, state_v = vmap(env.reset)(reset_keys)  # initiate envs
    traj_state = (bt, rng, state_v)  # initial state for step_fn

    for _ in tqdm(range(n_steps)):  # take n steps in env and append to lists
        obs_v, traj_state, state_v, reward_v = step(*traj_state, obs_v)  # take step
        state_seq, reward_seq = state_seq + [state_v], reward_seq + [reward_v]
        return

    return state_seq, reward_seq

In [18]:
# scenario=map_name_to_scenario(scenario))  # TODO: allow HeuristicSMAX
env = make("SMAX", num_allies=num_allies, num_enemies=num_enemies)
bt = jit(vmap(make_bt(env, bt_path), in_axes=(0, 0, None)))
rng, key = random.split(random.PRNGKey(0))
state_seq, reward_seq = traj_fn(bt, key, env)
# plot_fn(env, state_seq, reward_seq, expand=True)

  0%|          | 0/100 [00:00<?, ?it/s]


TypeError: Argument 'ally_0' of type <class 'str'> is not a valid JAX type

## language model